In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import tensorflow as tf
from scipy import misc
import cv2
import numpy as np
from library import facenet
from library import detect_face
import os
import time
import pickle

input_video="example_mov.avi"
modeldir = './model/20191012-185253.pb'
classifier_filename = './class/classifier.pkl'
npy='./npy'
train_img="./face_dataset"

with tf.Graph().as_default():
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.6)
    sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options, log_device_placement=False))
    with sess.as_default():
        pnet, rnet, onet = detect_face.create_mtcnn(sess, npy)

        minsize = 20  # minimum size of face
        threshold = [0.6, 0.7, 0.7]  # three steps's threshold
        factor = 0.709  # scale factor
        margin = 44
        frame_interval = 3
        batch_size = 1000
        image_size = 182
        input_image_size = 160
        
        HumanNames = os.listdir(train_img)
        HumanNames.sort()

        print('Loading Modal')
        facenet.load_model(modeldir)
        images_placeholder = tf.get_default_graph().get_tensor_by_name("input:0")
        embeddings = tf.get_default_graph().get_tensor_by_name("embeddings:0")
        phase_train_placeholder = tf.get_default_graph().get_tensor_by_name("phase_train:0")
        embedding_size = embeddings.get_shape()[1]


        classifier_filename_exp = os.path.expanduser(classifier_filename)
        with open(classifier_filename_exp, 'rb') as infile:
            (model, class_names) = pickle.load(infile)

        video_capture = cv2.VideoCapture(input_video)
        c = 0


        print('Start Recognition')
        prevTime = 0
        while True:
            ret, frame = video_capture.read()

            frame = cv2.resize(frame, (0,0), fx=0.5, fy=0.5)    #resize frame (optional)

            curTime = time.time()+1    # calc fps
            timeF = frame_interval

            if (c % timeF == 0):
                find_results = []

                if frame.ndim == 2:
                    frame = facenet.to_rgb(frame)
                frame = frame[:, :, 0:3]
                bounding_boxes, _ = detect_face.detect_face(frame, minsize, pnet, rnet, onet, threshold, factor)
                nrof_faces = bounding_boxes.shape[0]
                print('Detected_FaceNum: %d' % nrof_faces)

                if nrof_faces > 0:
                    det = bounding_boxes[:, 0:4]
                    img_size = np.asarray(frame.shape)[0:2]

                    cropped = []
                    scaled = []
                    scaled_reshape = []
                    bb = np.zeros((nrof_faces,4), dtype=np.int32)

                    for i in range(nrof_faces):
                        emb_array = np.zeros((1, embedding_size))

                        bb[i][0] = det[i][0]
                        bb[i][1] = det[i][1]
                        bb[i][2] = det[i][2]
                        bb[i][3] = det[i][3]

                        # inner exception
                        if bb[i][0] <= 0 or bb[i][1] <= 0 or bb[i][2] >= len(frame[0]) or bb[i][3] >= len(frame):
                            print('Face is very close!')
                            continue

                        cropped.append(frame[bb[i][1]:bb[i][3], bb[i][0]:bb[i][2], :])
                        cropped[i] = facenet.flip(cropped[i], False)
                        scaled.append(misc.imresize(cropped[i], (image_size, image_size), interp='bilinear'))
                        scaled[i] = cv2.resize(scaled[i], (input_image_size,input_image_size),
                                               interpolation=cv2.INTER_CUBIC)
                        scaled[i] = facenet.prewhiten(scaled[i])
                        scaled_reshape.append(scaled[i].reshape(-1,input_image_size,input_image_size,3))
                        feed_dict = {images_placeholder: scaled_reshape[i], phase_train_placeholder: False}
                        emb_array[0, :] = sess.run(embeddings, feed_dict=feed_dict)
                        predictions = model.predict_proba(emb_array)
                        print(predictions)
                        best_class_indices = np.argmax(predictions, axis=1)
                        best_class_probabilities = predictions[np.arange(len(best_class_indices)), best_class_indices]
                        # print("predictions")
                        print(best_class_indices,' with accuracy ',best_class_probabilities)

                        # print(best_class_probabilities)
                        if best_class_probabilities>0.53:
                            cv2.rectangle(frame, (bb[i][0], bb[i][1]), (bb[i][2], bb[i][3]), (0, 255, 0), 2)    #boxing face

                            #plot result idx under box
                            text_x = bb[i][0]
                            text_y = bb[i][3] + 20
                            print('Result Indices: ', best_class_indices[0])
                            print(HumanNames)
                            for H_i in HumanNames:
                                if HumanNames[best_class_indices[0]] == H_i:
                                    result_names = HumanNames[best_class_indices[0]]
                                    cv2.putText(frame, result_names, (text_x, text_y), cv2.FONT_HERSHEY_COMPLEX_SMALL,
                                                1, (0, 0, 255), thickness=1, lineType=2)
                else:
                    print('Alignment Failure')
            # c+=1
            cv2.imshow('Video', frame)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        video_capture.release()
        cv2.destroyAllWindows()

C:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Loading Modal
Model filename: ./model/20191012-185253.pb
Start Recognition
Detected_FaceNum: 3


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:98: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


[[0.02222524 0.08025056 0.05950839 0.05506129 0.02846647 0.75448805]]
[5]  with accuracy  [0.75448805]
Result Indices:  5
['Bill_Hader', 'Kate_McKinnon', 'bobby_moynihan', 'jason_sudeikis', 'kenan_thompson', 'kristen_wiig']
[[0.07736455 0.05107562 0.00794785 0.07519108 0.7280836  0.0603373 ]]
[4]  with accuracy  [0.7280836]
Result Indices:  4
['Bill_Hader', 'Kate_McKinnon', 'bobby_moynihan', 'jason_sudeikis', 'kenan_thompson', 'kristen_wiig']
[[0.16905698 0.14747242 0.19157955 0.16185728 0.11545911 0.21457465]]
[5]  with accuracy  [0.21457465]
Detected_FaceNum: 2
[[0.01138632 0.11502277 0.03108917 0.04104156 0.02484014 0.77662004]]
[5]  with accuracy  [0.77662004]
Result Indices:  5
['Bill_Hader', 'Kate_McKinnon', 'bobby_moynihan', 'jason_sudeikis', 'kenan_thompson', 'kristen_wiig']
[[0.04874054 0.04791258 0.00616544 0.10603753 0.75360839 0.03753552]]
[4]  with accuracy  [0.75360839]
Result Indices:  4
['Bill_Hader', 'Kate_McKinnon', 'bobby_moynihan', 'jason_sudeikis', 'kenan_thompson'

Detected_FaceNum: 1
[[0.00927861 0.04789071 0.01309195 0.02531849 0.01772385 0.88669639]]
[5]  with accuracy  [0.88669639]
Result Indices:  5
['Bill_Hader', 'Kate_McKinnon', 'bobby_moynihan', 'jason_sudeikis', 'kenan_thompson', 'kristen_wiig']
Detected_FaceNum: 1
[[0.00803618 0.02910591 0.01222234 0.02632504 0.01302321 0.91128731]]
[5]  with accuracy  [0.91128731]
Result Indices:  5
['Bill_Hader', 'Kate_McKinnon', 'bobby_moynihan', 'jason_sudeikis', 'kenan_thompson', 'kristen_wiig']
Detected_FaceNum: 1
[[0.00371817 0.01779521 0.00750817 0.01289961 0.00928603 0.9487928 ]]
[5]  with accuracy  [0.9487928]
Result Indices:  5
['Bill_Hader', 'Kate_McKinnon', 'bobby_moynihan', 'jason_sudeikis', 'kenan_thompson', 'kristen_wiig']
Detected_FaceNum: 1
[[0.00656797 0.03254762 0.02016366 0.01969375 0.02280319 0.89822381]]
[5]  with accuracy  [0.89822381]
Result Indices:  5
['Bill_Hader', 'Kate_McKinnon', 'bobby_moynihan', 'jason_sudeikis', 'kenan_thompson', 'kristen_wiig']
Detected_FaceNum: 1
[[0.0